## 교차검증 & 샘플링 & 그리드 서치
- 교차검증
  - 교차검증 != 정확도를 올린다
  - 교차검증의 조건 = Accuracy Stable Model(안정적인)

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
# input / target 1차 분리
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

## 세트 분리
- 기존) : train / test 세트
- 지금) : train / valid / test

In [ ]:
# train / test 8:2로 분리
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)
train_input.shape, test_input.shape, train_target.shape, test_target.shape

((5197, 3), (1300, 3), (5197,), (1300,))

In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

sub_input.shape, val_input.shape, sub_target.shape, val_target.shape

((4157, 3), (1040, 3), (4157,), (1040,))

- 모형 만들고, 모형 평가
  - 기존) test 데이터를 바로 평가 (unseen data : 미래에 올 데이터)
  - 지금) val 데이터만 평가

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt =  DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [ ]:
print(dt.score(test_input, test_target))

0.8569230769230769


## 교차검증

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)

print(scores)

{'fit_time': array([0.0121038 , 0.01054382, 0.01129436, 0.01120663, 0.01073456]), 'score_time': array([0.00164652, 0.00206113, 0.00140262, 0.00141835, 0.00147724]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
scores['test_score'] # -> 샘플링 결과가 편향되어있기 때문에 값의 차이가 다 다르다

array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])

In [ ]:
import numpy as np

# 검증 데이터 score(테스트데이터 score)
np.mean(scores['test_score'])

0.855300214703487

- StratfiedkFold 활용
  - 데이터를 분리 할때 덜 편향적으로 섞이게 하자
  - 통계용어 : 층화추출 (비율에 근거해서 추출하기) 

- 기존 방법 : 임의 추출 방식(무작위)

## 와인 데이터
- 유형 : 분류 모형
  - 분류기준 / 레드와인 , 화이트와인

In [ ]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
np.mean(scores['test_score'])

0.855300214703487

- StratifiedKFold() 세부 옵션 지정

In [ ]:
# n_splits : 몇번 교차 검증을 할건지
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
np.mean(scores['test_score'])

0.8574181117533719

## 하이퍼파라미터의 개념
- 기존) 수동으로 조정, 하나씩 값을 확인하는 형태
  - ridge(), Lasso(), alpha값 조정
  - decision tree, max_depth값 조정

- 현재 ) 자동화 개발 --> 머신러닝 엔지니어

In [ ]:
from sklearn.model_selection import GridSearchCV

# 모형 만들기
dt =  DecisionTreeClassifier(random_state = 42)

# 하이퍼 파라미터
params = {'max_depth' : [2,3,4,5,6,7]}
gs = GridSearchCV(dt, params, n_jobs=-1)

# 모형 학습 --> params의 인자의 개수만큼 수행
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7]})

- 가장 최적화된 max_depth 확인

In [ ]:
print(gs.best_params_)

{'max_depth': 5}


In [ ]:
best_dt = gs.best_estimator_
print(best_dt.score(train_input, train_target))

0.8672310948624207


In [ ]:
gs.cv_results_

{'mean_fit_time': array([0.00449448, 0.00482597, 0.00617032, 0.00758181, 0.0083365 ,
        0.00844827]),
 'std_fit_time': array([0.00089735, 0.00053887, 0.00077998, 0.00131504, 0.00100475,
        0.00076584]),
 'mean_score_time': array([0.00091138, 0.00126996, 0.0008688 , 0.00094838, 0.00088205,
        0.00090275]),
 'std_score_time': array([8.68716430e-05, 9.21038400e-04, 4.63695636e-05, 1.41978566e-04,
        1.78088305e-05, 4.20957288e-05]),
 'param_max_depth': masked_array(data=[2, 3, 4, 5, 6, 7],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2},
  {'max_depth': 3},
  {'max_depth': 4},
  {'max_depth': 5},
  {'max_depth': 6},
  {'max_depth': 7}],
 'split0_test_score': array([0.80480769, 0.84230769, 0.84711538, 0.84711538, 0.84807692,
        0.85769231]),
 'split1_test_score': array([0.8       , 0.83365385, 0.85384615, 0.86346154, 0.85480769,
        0.85192308]),
 'split2_test_score'

In [ ]:
gs.cv_results_['mean_test_score']

result = pd.DataFrame({
    'max_depth' : [2,3,4,5,6,7],
    'score' : gs.cv_results_['mean_test_score']
})

result

,max_depth,score
0,2,0.813936
1,3,0.841256
2,4,0.853378
3,5,0.857804
4,6,0.855880
5,7,0.855301


In [ ]:
# 여러 하이퍼 파라미터
params = {'max_depth' : [2,3,4,5,6,7],
          'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.01), # 노드 분할 시, 불순도 감소 최저량 지정
          'min_samples_split' : range(2, 100, 10)
          }

# 모형 만들기
dt =  DecisionTreeClassifier(random_state = 42)          
gs = GridSearchCV(dt, params, n_jobs=-1)# n_jobs = -1 -> 컴퓨터의 코어를 병렬처리하겠다 : 빠름

gs.fit(train_input, train_target)

print(gs.best_params_)

best_dt = gs.best_estimator_
print(best_dt.score(train_input, train_target))

{'max_depth': 7, 'min_impurity_decrease': 0.0001, 'min_samples_split': 92}
0.8793534731575909


## 랜덤 서치
- 매개변수 값의 목록을 전달하는 것이 아님
- 매개변수가 샘플링할 수있는 객체를 전달.

In [ ]:
from scipy.stats import uniform, randint
import numpy as np

# 고르게 추출하는 효과 - 정수
rgen = randint(0, 10)
np.unique(rgen.rvs(100), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 9, 14,  9, 10,  9, 10, 10, 14,  6,  9]))

In [ ]:
# 실수 무작위 추출
ugen = uniform(0,1)
ugen.rvs(10)

array([0.71870647, 0.43154225, 0.99934264, 0.8072642 , 0.47648894,
       0.91342099, 0.89633578, 0.51574634, 0.10079612, 0.25542662])

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# 랜덤 파라미터
params = {'max_depth' : randint(2,50),
          'min_impurity_decrease' : uniform(0.0001, 0.1),
          'min_samples_split' : randint(2,50)
          }

# 모형 만들기
dt =  DecisionTreeClassifier(random_state = 42)

# 랜덤 서치
rs = RandomizedSearchCV(dt, params, n_iter = 100 , random_state = 42 ,n_jobs = -1)

rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f13f277a490>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f13f28a8850>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f13f3d97450>},
                   random_state=42)

In [ ]:
print(rs.best_params_)

best_dt = rs.best_estimator_
print(best_dt.score(train_input, train_target))

{'max_depth': 39, 'min_impurity_decrease': 0.00017787658410143285, 'min_samples_split': 22}
0.9053299980758129


# 배운 것 리뷰


- 샘플링 중요 : 데이터가 편향적일 수 있다
- 과대적합 방지 : 하이퍼 파라미터 튜닝을 진행한다.
  + 그리드 서치
  + 랜덤 서치
  + 각 모델의 도움말을 잘 살펴본다.

## 오후 수업
- 트리의 앙상블 또는 다양한 트리 관련 모델 입문